In [2]:
import pandas as pd
import numpy as np

data =  pd.read_csv('./aave_barrow_contract.csv')

df = data.copy()

df.head(3)

,Txhash,Blockno,UnixTimestamp,DateTime,From,To,ContractAddress,Value_IN(ETH),Value_OUT(ETH),CurrentValue @ $1869.43/Eth,TxnFee(ETH),TxnFee(USD),Historical $Price/Eth,Method
0,0xa23b260fa3ec2677889587a5bc72e843ae6f39e5adf1...,22104765,1662458272,06/09/2022 09:57,0x4365f8e70cf38c6ca67de41448508f2da8825500,NaN,0x76d3030728e52deb8848d5613abade88441cbc59,0.0,0,0.0,0.000730,1.364804,1558.9,0x60c06040
1,0x64b50ed929f06e574dc0dbfbaae11905e171878b15af...,22154621,1662483310,06/09/2022 16:55,0xd53c79ff8c473bbfe4e40e5525d4d24fd4b8534c,0x76d3030728e52deb8848d5613abade88441cbc59,NaN,0.0,0,0.0,0.000097,0.180834,1558.9,Withdraw ETH
2,0x650a432506ecd60e6b968469f9e09f9d35826e33015d...,22154653,1662483341,06/09/2022 16:55,0xd53c79ff8c473bbfe4e40e5525d4d24fd4b8534c,0x76d3030728e52deb8848d5613abade88441cbc59,NaN,0.0,0,0.0,0.000103,0.192614,1558.9,Withdraw ETH


Merged Dataframe:
                                         Address  Value_IN(ETH)_x  \
0     0x0000000a5d2afc64886a80780bc5ca6ecc519181         0.000005   
1     0x0000000f3a9e446d0a6f976df5f0ed1b324dcd0e         0.000005   
2     0x00002b503a75998c97508916a74fdb41934fa030        64.800000   
3     0x0000ce08fa224696a819877070bf378e8b131acf         0.001000   
4     0x0026c9a5bfbee977ad03dcb4a95cfa715278e779         0.012160   
...                                          ...              ...   
1855  0xfc1d987a42a0e37bd19ae7bc8d7c0566b8555d9c              NaN   
1856  0xfcc1b0aaeb2b9063ab34fdfbb3eedc2ff30ae4f2              NaN   
1857  0xfd8a8ff896ccd28efe1c4a3ef69b9c4fd3f1943d              NaN   
1858  0xffb3468b6861ba7044fcf61a2d976028204efc49              NaN   
1859  0xffc57ab280cb8d1e47e11ce225f113612cff3d8e              NaN   

     Deposit Value Score        DateTime_x Deposit Time Score Value_OUT(ETH)  \
0                     10  09/09/2022 03:35                 70            

In [27]:
import pandas as pd

# Read the dataset into a pandas DataFrame

# Define the scoring criteria and intervals
deposit_value_intervals = [0, 1, 10, 100, 1000, 10000]
deposit_value_scores = [10, 25, 40, 55, 70, 100]

deposit_time_intervals = [0, 30, 90, 180, 365]
deposit_time_scores = [10, 30, 50, 70, 100]

borrow_value_intervals = [0, 1, 10, 100, 1000, 10000]
borrow_value_scores = [10, 25, 40, 55, 70, 100]

borrow_time_intervals = [0, 30, 90, 180, 365]
borrow_time_scores = [10, 30, 50, 70, 100]

repayment_value_intervals = [0, 1, 10, 100, 1000, 10000]
repayment_value_scores = [10, 25, 40, 55, 70, 100]

# Create empty dataframes for each category
deposit_df = pd.DataFrame(columns=['Address', 'Value_IN(ETH)', 'Deposit Value Score'])
deposit_time_df = pd.DataFrame(columns=['Address', 'DateTime', 'Deposit Time Score'])
borrow_df = pd.DataFrame(columns=['Address', 'Value_OUT(ETH)', 'Borrow Value Score'])
borrow_time_df = pd.DataFrame(columns=['Address', 'DateTime', 'Borrow Time Score'])
repayment_df = pd.DataFrame(columns=['Address', 'Value_IN(ETH)', 'Repayment Value Score'])

# Iterate over each unique address in the dataset
for address, group_data in df.groupby('From'):
    methods = group_data['Method'].unique()
    for method in methods:
        if 'Deposit ETH' in method:
            value_in_eth = group_data['Value_IN(ETH)'].sum()
            deposit_value_score = None
            deposit_time_score = None

            # Calculate the score for deposit value
            if 0 <= value_in_eth < 1:
                deposit_value_score = 10
            elif 1 <= value_in_eth < 10:
                deposit_value_score = 25
            elif 10 <= value_in_eth < 100:
                deposit_value_score = 40
            elif 100 <= value_in_eth < 1000:
                deposit_value_score = 55
            elif 1000 <= value_in_eth < 10000:
                deposit_value_score = 70
            else:
                deposit_value_score = 100

            deposit_df = pd.concat([deposit_df, pd.DataFrame({'Address': [address], 'Value_IN(ETH)': [value_in_eth], 'Deposit Value Score': [deposit_value_score]})], ignore_index=True)

            # Calculate the score for deposit time
            days_since_deposit = (pd.Timestamp.now() - pd.to_datetime(group_data['DateTime'])).dt.days
            if days_since_deposit.iloc[0] <= 30:
                deposit_time_score = 10
            elif days_since_deposit.iloc[0] <= 90:
                deposit_time_score = 30
            elif days_since_deposit.iloc[0] <= 180:
                deposit_time_score = 50
            elif days_since_deposit.iloc[0] <= 365:
                deposit_time_score = 70
            else:
                deposit_time_score = 100

            deposit_time_df = pd.concat([deposit_time_df, pd.DataFrame({'Address': [address], 'DateTime': [group_data['DateTime'].iloc[0]], 'Deposit Time Score': [deposit_time_score]})], ignore_index=True)

        elif 'Borrow ETH' in method:
            value_out_eth = group_data['Value_OUT(ETH)'].sum()
            borrow_value_score = None
            borrow_time_score = None

            # Calculate the score for borrow value
            if 0 <= value_out_eth < 1:
                borrow_value_score = 10
            elif 1 <= value_out_eth < 10:
                borrow_value_score = 25
            elif 10 <= value_out_eth < 100:
                borrow_value_score = 40
            elif 100 <= value_out_eth < 1000:
                borrow_value_score = 55
            elif 1000 <= value_out_eth < 10000:
                borrow_value_score = 70
            else:
                borrow_value_score = 100

            borrow_df = pd.concat([borrow_df, pd.DataFrame({'Address': [address], 'Value_OUT(ETH)': [value_out_eth], 'Borrow Value Score': [borrow_value_score]})], ignore_index=True)

            # Calculate the score for borrow time
            days_since_borrow = (pd.Timestamp.now() - pd.to_datetime(group_data['DateTime'])).dt.days
            if days_since_borrow.iloc[0] <= 30:
                borrow_time_score = 10
            elif days_since_borrow.iloc[0] <= 90:
                borrow_time_score = 30
            elif days_since_borrow.iloc[0] <= 180:
                borrow_time_score = 50
            elif days_since_borrow.iloc[0] <= 365:
                borrow_time_score = 70
            else:
                borrow_time_score = 100

            borrow_time_df = pd.concat([borrow_time_df, pd.DataFrame({'Address': [address], 'DateTime': [group_data['DateTime'].iloc[0]], 'Borrow Time Score': [borrow_time_score]})], ignore_index=True)

        elif 'Repay ETH' in method:
            value_in_eth = group_data['Value_IN(ETH)'].sum()
            repayment_value_score = None

            # Calculate the score for repayment value
            if 0 <= value_in_eth < 1:
                repayment_value_score = 10
            elif 1 <= value_in_eth < 10:
                repayment_value_score = 25
            elif 10 <= value_in_eth < 100:
                repayment_value_score = 40
            elif 100 <= value_in_eth < 1000:
                repayment_value_score = 55
            elif 1000 <= value_in_eth < 10000:
                repayment_value_score = 70
            else:
                repayment_value_score = 100

            repayment_df = pd.concat([repayment_df, pd.DataFrame({'Address': [address], 'Value_IN(ETH)': [value_in_eth], 'Repayment Value Score': [repayment_value_score]})], ignore_index=True)

# Merge all dataframes based on the 'Address' column
lp_scores_df = pd.merge(deposit_df, deposit_time_df, on='Address', how='outer')
lp_scores_df = pd.merge(lp_scores_df, borrow_df, on='Address', how='outer')
lp_scores_df = pd.merge(lp_scores_df, borrow_time_df, on='Address', how='outer')
lp_scores_df = pd.merge(lp_scores_df, repayment_df, on='Address', how='outer')

# Calculate LP scores for each unique address by summing up all the scores
lp_scores_df['LP Score'] = lp_scores_df[['Deposit Value Score', 'Deposit Time Score', 'Borrow Value Score', 'Borrow Time Score', 'Repayment Value Score']].sum(axis=1)

# Calculate LP rewards for each unique address
lp_scores_df['LP Reward'] = (lp_scores_df['LP Score'] / 500) * 100

# Print the merged dataframe
print("Merged Dataframe:")
print(lp_scores_df)


Merged Dataframe:
                                         Address  Value_IN(ETH)_x  \
0     0x0000000a5d2afc64886a80780bc5ca6ecc519181         0.000005   
1     0x0000000f3a9e446d0a6f976df5f0ed1b324dcd0e         0.000005   
2     0x00002b503a75998c97508916a74fdb41934fa030        64.800000   
3     0x0000ce08fa224696a819877070bf378e8b131acf         0.001000   
4     0x0026c9a5bfbee977ad03dcb4a95cfa715278e779         0.012160   
...                                          ...              ...   
1983  0xf576dae6f10ea36a4dfade275aa576560e3b4b4a              NaN   
1984  0xfbd08fd2b2c7148b4e38624158c3155b5b49f886              NaN   
1985  0xfc1d987a42a0e37bd19ae7bc8d7c0566b8555d9c              NaN   
1986  0xfd8a8ff896ccd28efe1c4a3ef69b9c4fd3f1943d              NaN   
1987  0xffc57ab280cb8d1e47e11ce225f113612cff3d8e              NaN   

     Deposit Value Score        DateTime_x Deposit Time Score Value_OUT(ETH)  \
0                     10  09/09/2022 03:35                 70            

In [32]:
#lp_scores_df = lp_scores_df.drop(columns=['DateTime_x', 'DateTime_y'])

# Replace NaN values with 0 in the lp_scores_df
lp_scores_df = lp_scores_df.fillna(0)

lp_scores_df.head(3)

,Address,Value_IN(ETH)_x,Deposit Value Score,Deposit Time Score,Value_OUT(ETH),Borrow Value Score,Borrow Time Score,Value_IN(ETH)_y,Repayment Value Score,LP Score,LP Reward
0,0x0000000a5d2afc64886a80780bc5ca6ecc519181,0.000005,10,70,0,0,0,0.0,0,80.0,16.0
1,0x0000000f3a9e446d0a6f976df5f0ed1b324dcd0e,0.000005,10,70,0,0,0,0.0,0,80.0,16.0
2,0x00002b503a75998c97508916a74fdb41934fa030,64.800000,40,70,0,0,0,0.0,0,110.0,22.0


In [33]:
lp_scores = lp_scores_df.copy()

In [34]:
lp_scores_df.head(3)

,Address,Value_IN(ETH)_x,Deposit Value Score,Deposit Time Score,Value_OUT(ETH),Borrow Value Score,Borrow Time Score,Value_IN(ETH)_y,Repayment Value Score,LP Score,LP Reward
0,0x0000000a5d2afc64886a80780bc5ca6ecc519181,0.000005,10,70,0,0,0,0.0,0,80.0,16.0
1,0x0000000f3a9e446d0a6f976df5f0ed1b324dcd0e,0.000005,10,70,0,0,0,0.0,0,80.0,16.0
2,0x00002b503a75998c97508916a74fdb41934fa030,64.800000,40,70,0,0,0,0.0,0,110.0,22.0


In [37]:

aa = df[df['From']=='0x00002b503a75998c97508916a74fdb41934fa030']

aa['Value_IN(ETH)'].sum()

64.8

In [38]:

lp_scores.sort_values(by='Value_IN(ETH)_y', ascending=False)

,Address,Value_IN(ETH)_x,Deposit Value Score,Deposit Time Score,Value_OUT(ETH),Borrow Value Score,Borrow Time Score,Value_IN(ETH)_y,Repayment Value Score,LP Score,LP Reward
401,0x360537542135943e8fc1562199aea6d0017f104b,6805.001707,70,70,0,10,70,6805.001707,70,290.0,58.0
1147,0xa3fdc58439b4677a11b9b0c49cae0fca9c23ab8a,5734.400000,70,70,0,10,70,5734.400000,70,290.0,58.0
1844,0x5c311563ea0bb8f9ca2471012eb2f495bd687d18,0.000000,0,0,0,10,100,4858.883837,70,180.0,36.0
1224,0xaf8f0b2b7dee0dc067d88e1a1c8e284be87871fb,2104.578892,70,70,0,0,0,2104.578892,70,210.0,42.0
1653,0xeb18c039ce0be7b26f872d9847bf94a649aab797,733.686591,55,70,0,10,70,733.686591,55,260.0,52.0
...,...,...,...,...,...,...,...,...,...,...,...
664,0x5b558496fa9f93942974740d6609da174e489ad5,1.100000,25,70,0,0,0,0.000000,0,95.0,19.0
663,0x5b2ed2ef8f480cc165a600ac451d9d9ebf521e94,1.165494,25,70,0,0,0,0.000000,0,95.0,19.0
662,0x5ae92f515e2505f8dec1e61355f1533bb8627090,18.200000,40,70,0,0,0,0.000000,0,110.0,22.0
661,0x5ab4e386af96e8d493f0f1bc1e214eb108418d97,0.012700,10,70,0,0,0,0.000000,0,80.0,16.0


In [39]:
df[df['From']=='0x5c311563ea0bb8f9ca2471012eb2f495bd687d18']

,Txhash,Blockno,UnixTimestamp,DateTime,From,To,ContractAddress,Value_IN(ETH),Value_OUT(ETH),CurrentValue @ $1869.43/Eth,TxnFee(ETH),TxnFee(USD),Historical $Price/Eth,Method
440,0xa88f2720512cbcad313abc2202eabe9322bbd074d1bf...,22293766,1662571087,07/09/2022 17:18,0x5c311563ea0bb8f9ca2471012eb2f495bd687d18,0x76d3030728e52deb8848d5613abade88441cbc59,NaN,0.000000,0,0.000,0.000101,0.188300,1630.00,Borrow ETH
807,0x048b8d5b10af0cf9cfc5c0803c912e73e7a7f4e548fc...,22419990,1662656769,08/09/2022 17:06,0x5c311563ea0bb8f9ca2471012eb2f495bd687d18,0x76d3030728e52deb8848d5613abade88441cbc59,NaN,0.000000,0,0.000,0.000184,0.344540,1635.60,Borrow ETH
809,0x75d85bb9cbe189c3d23e56ab15942e8f216e16ae4eec...,22420845,1662657361,08/09/2022 17:16,0x5c311563ea0bb8f9ca2471012eb2f495bd687d18,0x76d3030728e52deb8848d5613abade88441cbc59,NaN,0.000000,0,0.000,0.000096,0.178660,1635.60,Borrow ETH
819,0x19a2241d7f09f574ab14dc4df574ecdb6207274311dd...,22424478,1662660455,08/09/2022 18:07,0x5c311563ea0bb8f9ca2471012eb2f495bd687d18,0x76d3030728e52deb8848d5613abade88441cbc59,NaN,0.000000,0,0.000,0.000155,0.289804,1635.60,Borrow ETH
3499,0xae98f6316a4026ab244df5eace45aa6f935dea61f336...,23245290,1663231836,15/09/2022 08:50,0x5c311563ea0bb8f9ca2471012eb2f495bd687d18,0x76d3030728e52deb8848d5613abade88441cbc59,NaN,4858.883837,0,9083343.212,0.000062,0.115401,1472.51,Repay ETH


In [42]:

lp_new = lp_scores_df.copy()

In [43]:

# Iterate over each row in the dataframe
for index, row in lp_new.iterrows():
    # Get the values in 'Value_IN(ETH)_x' and 'Value_IN(ETH)_y'
    value_x = row['Value_IN(ETH)_x']
    value_y = row['Value_IN(ETH)_y']
    
    # Check if 'Value_IN(ETH)_x' is 0 and replace it with 'Value_IN(ETH)_y'
    if value_x == 0:
        lp_new.at[index, 'Value_IN(ETH)_x'] = value_y

# Drop the 'Value_IN(ETH)_y' column as it's no longer needed
lp_new.drop(columns=['Value_IN(ETH)_y'], inplace=True)


In [44]:
lp_new.head(3)

,Address,Value_IN(ETH)_x,Deposit Value Score,Deposit Time Score,Value_OUT(ETH),Borrow Value Score,Borrow Time Score,Repayment Value Score,LP Score,LP Reward
0,0x0000000a5d2afc64886a80780bc5ca6ecc519181,0.000005,10,70,0,0,0,0,80.0,16.0
1,0x0000000f3a9e446d0a6f976df5f0ed1b324dcd0e,0.000005,10,70,0,0,0,0,80.0,16.0
2,0x00002b503a75998c97508916a74fdb41934fa030,64.800000,40,70,0,0,0,0,110.0,22.0


In [46]:

lp_new[lp_new['Address']=='0x5c311563ea0bb8f9ca2471012eb2f495bd687d18']

,Address,Value_IN(ETH)_x,Deposit Value Score,Deposit Time Score,Value_OUT(ETH),Borrow Value Score,Borrow Time Score,Repayment Value Score,LP Score,LP Reward
1844,0x5c311563ea0bb8f9ca2471012eb2f495bd687d18,4858.883837,0,0,0,10,100,70,180.0,36.0


In [47]:
lp_new.columns

Index(['Address', 'Value_IN(ETH)_x', 'Deposit Value Score',
       'Deposit Time Score', 'Value_OUT(ETH)', 'Borrow Value Score',
       'Borrow Time Score', 'Repayment Value Score', 'LP Score', 'LP Reward'],
      dtype='object')